In [ ]:
import numpy as np


# 関数を改良
class Variable:
    def __init__(self, data):
        # 追加
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError("{} is not supported".format(type(data)))
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        # 追加
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)

class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(as_array(y))
        output.set_creator(self)
        self.input = input
        self.output = output
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()


class Square(Function):
    def forward(self, x):
        return x ** 2

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

class Exp(Function):
    def forward(self, x):
        return np.exp(x)

    def backward(self, gy):
        return np.exp(self.input.data) * gy


# 追加
def square(x):
    return Square()(x)

def exp(x):
    return Exp()(x)

# 独自の関数を用意
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    else:
        return x

In [7]:
# 順伝搬テスト
import unittest

class SquareTest(unittest.TestCase):
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected)

In [8]:
# 成功例
!python3 -m unittest step10.py

.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


In [5]:
# 失敗例
!python3 -m unittest step10.py

F
FAIL: test_forward (step10.SquareTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/Users/nakamurataiki/Desktop/DeZero/steps/step10.py", line 88, in test_forward
    self.assertEqual(y.data, expected)
AssertionError: array(4.) != array(5.)

----------------------------------------------------------------------
Ran 1 test in 0.001s

FAILED (failures=1)


In [9]:
# 逆伝播テスト
# 順伝搬テスト
import unittest

class SquareTest(unittest.TestCase):
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected)

    def test_backward(self):
        x = Variable(np.array(3.0))
        y = square(x)
        y.backward()
        expected = np.array(6.0)
        self.assertEqual(x.grad, expected)

In [10]:
!python3 -m unittest step10.py

..
----------------------------------------------------------------------
Ran 2 tests in 0.000s

OK


In [11]:
# 勾配確認による自動テスト
def numerical_diff(f, x, eps=1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2 * eps)

In [ ]:
# 逆伝播テスト(自動)
# 順伝搬テスト
import unittest

class SquareTest(unittest.TestCase):
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected)

    def test_backward(self):
        x = Variable(np.array(3.0))
        y = square(x)
        y.backward()
        expected = np.array(6.0)
        self.assertEqual(x.grad, expected)

    def test_gradient_check(self):
        # ランダムな入力値を生成
        x = Variable(np.random.rand(1))
        y = square(x)
        y.backward()
        num_grad = numerical_diff(square, x)
        flg = np.allclose(x.grad, num_grad)
        self.assertTrue(flg)

In [13]:
!python3 -m unittest step10.py

...
----------------------------------------------------------------------
Ran 3 tests in 0.001s

OK
